In [1]:
#load the training and testing datasets

def load_datasets(trainFile, testFile):
    
    import pandas as pd
    trainDataset = pd.read_csv(trainFile)
    testDataset = pd.read_csv(testFile)
    
    trainDataset.drop(trainDataset.columns[[0]], axis = 1, inplace = True)
    testDataset.drop(testDataset.columns[[0]], axis = 1, inplace = True)
    
    return trainDataset, testDataset


In [2]:
#seperating test and train into features and target

def prepare_datasets(trainDataset, testDataset):
    
    #selecting first 13 columns
    trainX = trainDataset[trainDataset.columns[0:15]]
    trainY = trainDataset[trainDataset.columns[15:16]]

    testX = testDataset[testDataset.columns[0:15]]
    testY = testDataset[testDataset.columns[15:16]]

    return trainX, trainY, testX, testY

In [3]:
#built for neural network
def create_model(optimizer = 'adam', batch_size = 10, epochs = 10, learn_rate = 0.001, momentum = 0.0, init_mode = 'uniform', weight_constraint = 1 ):
    
    import numpy
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.wrappers.scikit_learn import KerasClassifier
    
    model = Sequential()
    model.add(Dense(15, input_dim=8, activation='relu'))
    model.add(Dense(15, activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    
    return model

In [4]:
#built for neural network
def baselineMLP(DataTrainX, DataTrainY, DataTestX, DataTestY):
    
    from keras.wrappers.scikit_learn import KerasClassifier
    import numpy
    
    seed = 7
    numpy.random.seed(seed)
                      
    kerasModel = KerasClassifier(build_fn = create_model)
    kerasModel.fit(DataTrainX, DataTrainY, epochs=100, batch_size=60)
    
    predictedData = kerasModel.predict(DataTestX)
    
    predictedData[predictedData >= 0.5] = 1
    predictedData[predictedData < 0.5] = 0

    print("The performance of the MLP is :")
    measure_network(predictedData, DataTestY)
        

The performance of the MLP is : - 15 features
Accuracy Score : 0.5885374425520411
Precision Score : 0.7454873646209387
Recall Score : 0.2302118171683389
F1 Score : 0.35178875638841567
The MSE is : 0.4114625574479589

The performance of the MLP is : - 9 features
Accuracy Score : 0.6226007028926737
Precision Score : 0.559322033898305
Recall Score : 0.22510231923601637
F1 Score : 0.321011673151751
The MSE is : 0.3773992971073263

The performance of the MLP is : - 6 features
Accuracy Score : 0.5304136253041363
Precision Score : 0.7160493827160493
Recall Score : 0.20484359233097882
F1 Score : 0.3185562965868968
The MSE is : 0.46958637469586373

In [5]:
#baseline RF implementation
def baselineRF(DataTrainX, DataTrainY, DataTestX, DataTestY):
    
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(n_estimators = 1000, random_state = 42, verbose=100)
    rf.fit(DataTrainX, DataTrainY)
    predictedData = rf.predict(DataTestX)
    
    print("The performance of the Baseline RF model is :")
    measure_network(predictedData, DataTestY)




The performance of the Baseline RF model is : - 15 features
Accuracy Score : 0.7493917274939172
Precision Score : 0.26353790613718414
Recall Score : 0.21954887218045113
F1 Score : 0.2395406070549631
The MSE is : 0.25060827250608275


The performance of the Baseline RF model is : - 9 features
Accuracy Score : 0.6193565828602325
Precision Score : 0.5610169491525424
Recall Score : 0.22364864864864864
F1 Score : 0.31980676328502416
The MSE is : 0.3806434171397675


The performance of the Baseline RF model is : - 6 features
Accuracy Score : 0.5293322519599892
Precision Score : 0.7178130511463845
Recall Score : 0.20472837022132798
F1 Score : 0.31859099804305285
The MSE is : 0.4706677480400108

In [6]:
def baselineSVM(DataTrainX, DataTrainY, DataTestX, DataTestY):
    
    import pandas as pd 
    import numpy as np
    from sklearn.svm import SVC 
    model = SVC(verbose = 100) 
    model.fit(DataTrainX, DataTrainY)
    predictedData = model.predict(DataTestX)
    
    predictedData[predictedData >= 0.5] = 1
    predictedData[predictedData < 0.5] = 0
    
    print("The performance of the Baseline SVM model is :")
    measure_network(predictedData, DataTestY)


    

Baseline SVM - 15 features
Accuracy Score : 0.5479859421465261
Precision Score : 0.7635379061371841
Recall Score : 0.21537678207739308
F1 Score : 0.3359809372517871
The MSE is : 0.4520140578534739

The performance of the Baseline SVM model is : - 9 features
Accuracy Score : 0.6188158961881589
Precision Score : 0.559322033898305
Recall Score : 0.22297297297297297
F1 Score : 0.31884057971014496
The MSE is : 0.38118410381184104

The performance of the Baseline SVM model is : - 6 featyres
Accuracy Score : 0.5296025952960259
Precision Score : 0.7178130511463845
Recall Score : 0.20483140412682435
F1 Score : 0.3187157400156617
The MSE is : 0.47039740470397406

In [7]:
def bestRF(DataTrainX, DataTrainY, DataTestX, DataTestY):
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    
    param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
    }
    
    rf = RandomForestClassifier(verbose=100)
    met_grid= ['accuracy', 'f1']
    grid = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 100, scoring = met_grid, refit = 'accuracy')
    grid.fit(DataTrainX, DataTrainY)
    predictedData = grid.predict(DataTestX)
    
    print("The performance of the Best RF model is :")
    measure_network(predictedData, DataTestY)
    
    print("The best parameters are : ")
    print(grid.best_params_)


The performance of the Best RF model is : - 6 features
Accuracy Score : 0.8672614220059476
Precision Score : 0.7631578947368421
Recall Score : 0.549936788874842
F1 Score : 0.639235855988244
The MSE is : 0.13273857799405245
The best parameters are : 
{'bootstrap': True, 'max_depth': 110, 'max_features': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 1000}


The performance of the Best RF model is : - 9 features
Accuracy Score : 0.8537442552041092
Precision Score : 0.7770154373927959
Recall Score : 0.5243055555555556
F1 Score : 0.626123013130615
The MSE is : 0.1462557447958908
The best parameters are : 
{'bootstrap': True, 'max_depth': 90, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}


The performance of the Best RF model is : - 13 features
Accuracy Score : 0.8848337388483374
Precision Score : 0.7934990439770554
Recall Score : 0.5661664392905866
F1 Score : 0.6608280254777069
The MSE is : 0.1151662611516626
The best parameters are : 
{'bootstrap': True, 'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 100}
​​

In [9]:
def measure_network(prediction, actual):

    from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score, mean_squared_error
    print('Accuracy Score : ' + str(accuracy_score(prediction, actual)))
    print('Precision Score : ' + str(precision_score(prediction, actual)))
    print('Recall Score : ' + str(recall_score(prediction, actual)))
    print('F1 Score : ' + str(f1_score(prediction, actual)))
    print("The MSE is : " + str(mean_squared_error(prediction, actual)))
    

In [10]:
trainData, testData = load_datasets("TrainingData.csv", "TestingData.csv")
if(trainData.empty==False):
    print("Training data is loaded")
if(testData.empty==False):
    print("Testsing data is loaded")
    

trainDataX, trainDataY, testDataX, testDataY = prepare_datasets(trainData, testData)
if(trainDataX.size!=0):
    print("Training data X has been created successfully")
if(trainDataY.size!=0):
    print("Training data Y has been created successfully")
if(testDataX.size!=0):
    print("Testing data X has been created successfully")
if(testDataY.size!=0):
    print("Testing data Y has been created successfully")
    
    
#baselineMLP(trainDataX, trainDataY, testDataX, testDataY)
#baselineRF(trainDataX, trainDataY, testDataX, testDataY)
#baselineSVM(trainDataX, trainDataY, testDataX, testDataY)

bestRF(trainDataX, trainDataY, testDataX, testDataY)
#bestMLP(trainDataX, trainDataY, testDataX, testDataY)



Training data is loaded
Testsing data is loaded
Training data X has been created successfully
Training data Y has been created successfully
Testing data X has been created successfully
Testing data Y has been created successfully
Fitting 3 folds for each of 288 candidates, totalling 864 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to new file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-2608180895560-3db8da0ce19f4c0e807b1a997bd40a20.pkl
Pickling array (shape=(5, 14492), dtype=float64).
Pickling array (shape=(10,), dtype=object).
Pickling array (shape=(5,), dtype=object).
Pickling array (shape=(10,), dtype=int64).
Pickling array (shape=(5,), dtype=int64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(1, 14492), dtype=float64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(9661,), dtype=i

[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    3.5s
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-2608180895560-3db8da0ce19f4c0e807b1a997bd40a20.pkl
Pickling array (shape=(5, 14492), dtype=float64).
Pickling array (shape=(10,), dtype=object).
Pickling array (shape=(5,), dtype=object).
Pickling array (shape=(10,), dtype=int64).
Pickling array (shape=(5,), dtype=int64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(1, 14492), dtype=float64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(9661,), dtype=int32).
Pickling array (shape=(4831,), dtype=int32).
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\

[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    8.8s
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-2608180895560-3db8da0ce19f4c0e807b1a997bd40a20.pkl
Pickling array (shape=(5, 14492), dtype=float64).
Pickling array (shape=(10,), dtype=object).
Pickling array (shape=(5,), dtype=object).
Pickling array (shape=(10,), dtype=int64).
Pickling array (shape=(5,), dtype=int64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(1, 14492), dtype=float64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(9661,), dtype=int32).
Pickling array (shape=(4831,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   10.1s
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-260818089

[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:   16.5s
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-2608180895560-3db8da0ce19f4c0e807b1a997bd40a20.pkl
Pickling array (shape=(5, 14492), dtype=float64).
Pickling array (shape=(10,), dtype=object).
Pickling array (shape=(5,), dtype=object).
Pickling array (shape=(10,), dtype=int64).
Pickling array (shape=(5,), dtype=int64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(1, 14492), dtype=float64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(9662,), dtype=int32).
Pickling array (shape=(4830,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   17.7s
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-260818089

[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   24.8s
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-2608180895560-3db8da0ce19f4c0e807b1a997bd40a20.pkl
Pickling array (shape=(5, 14492), dtype=float64).
Pickling array (shape=(10,), dtype=object).
Pickling array (shape=(5,), dtype=object).
Pickling array (shape=(10,), dtype=int64).
Pickling array (shape=(5,), dtype=int64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(1, 14492), dtype=float64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(9661,), dtype=int32).
Pickling array (shape=(4831,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:   25.0s
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-260818089

...

[Parallel(n_jobs=-1)]: Done 836 tasks      | elapsed: 10.2min
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-2608180895560-3db8da0ce19f4c0e807b1a997bd40a20.pkl
Pickling array (shape=(5, 14492), dtype=float64).
Pickling array (shape=(10,), dtype=object).
Pickling array (shape=(5,), dtype=object).
Pickling array (shape=(10,), dtype=int64).
Pickling array (shape=(5,), dtype=int64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(1, 14492), dtype=float64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(9661,), dtype=int32).
Pickling array (shape=(4831,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 837 tasks      | elapsed: 10.2min
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-260818089

[Parallel(n_jobs=-1)]: Done 848 tasks      | elapsed: 10.3min
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-2608180895560-3db8da0ce19f4c0e807b1a997bd40a20.pkl
Pickling array (shape=(5, 14492), dtype=float64).
Pickling array (shape=(10,), dtype=object).
Pickling array (shape=(5,), dtype=object).
Pickling array (shape=(10,), dtype=int64).
Pickling array (shape=(5,), dtype=int64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(1, 14492), dtype=float64).
Pickling array (shape=(1,), dtype=object).
Pickling array (shape=(9661,), dtype=int32).
Pickling array (shape=(4831,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 849 tasks      | elapsed: 10.3min
Pickling array (shape=(15,), dtype=object).
Memmapping (shape=(10, 14492), dtype=int64) to old file C:\Users\Pallavi\AppData\Local\Temp\joblib_memmapping_folder_21496_2570550538\21496-260818089

C:\MyProgramFiles\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.1s remaining:    0.0s
building tree 22 of 100
[Parallel(n_jobs=1)]: Done  22 out of  22 | elapsed:    0.1s remaining:    0.0s
building tree 23 of 100
[Parallel(n_jobs=1)]: Done  23 out of  23 | elapsed:    0.1s remaining:    0.0s
building tree 24 of 100
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.1s remaining:    0.0s
building tree 25 of 100
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    0.1s remaining:    0.0s
building tree 26 of 100
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:    0.1s remaining:    0.0s
building tree 27 of 100
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:    0.1s remaining:    0.0s
building tree 28 of 100
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:    0.1s remaining:    0.0s
building tree 29 of 100
[Parallel(n_jobs=1)]: Done  29 out of  29 | elapsed:    0.1s remaining:    0.0s
building tree 30 of 100
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:    0.1s rem

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elaps